## Algoritmo Random Forest
- Previsão do posicionamento da Abelha Trigona Spinipes no período de 2021-2040, 2041-2060, 2061-80 e 2081-2100

### Configuração de Ambiente 


In [73]:
# Instalar dependências necessárias
# pip install pandas geopandas rasterio scikit-learn matplotlib streamlit folium streamlit-folium

In [74]:
# Importar bibliotecas (adicionando pathlib)
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.plot import show
from rasterio.features import geometry_mask
import glob
import os
from pathlib import Path  # <<< IMPORTAMOS A BIBLIOTECA PATHLIB
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import warnings

# Ignorar avisos para uma saída mais limpa
warnings.filterwarnings('ignore')

# --- DEFINIÇÃO AUTOMÁTICA DOS CAMINHOS ---
# Esta linha mágica encontra o caminho da pasta do projeto automaticamente!
# Path.cwd() pega o diretório atual (.../abelhas_extensao/notebooks)
# .parent sobe um nível (.../abelhas_extensao)
BASE_DIR = Path.cwd().parent

# Definir os caminhos completos usando o operador / do pathlib
OCORRENCIAS_PATH = BASE_DIR / 'data' / 'ocorrencias.csv'
CLIMA_ATUAL_PATH = BASE_DIR / 'data' / 'clima_atual'
BRASIL_SHAPE_PATH = BASE_DIR / 'data' / 'BR_UF_2024'
PASTA_PREVISOES = BASE_DIR / 'data' / 'previsoes_futuras'

# Criar a pasta para salvar as previsões, se ela não existir
PASTA_PREVISOES.mkdir(parents=True, exist_ok=True)

print("Ambiente configurado e caminhos definidos automaticamente com pathlib!")
print(f"Pasta raiz do projeto encontrada: {BASE_DIR}")
print(f"Caminho do arquivo de ocorrências: {OCORRENCIAS_PATH}")

Ambiente configurado e caminhos definidos automaticamente com pathlib!
Pasta raiz do projeto encontrada: c:\icev\extensao\abelhas_extensao
Caminho do arquivo de ocorrências: c:\icev\extensao\abelhas_extensao\data\ocorrencias.csv


### Tratamento e Carregamento dos Dados


#### Carregar Dados de Ocorrência e Mapa do Brasil

In [75]:
# Carregar dados de ocorrência de forma robusta (VERSÃO FINAL CORRIGIDA)
try:
    print("Tentando carregar o arquivo 'ocorrencias.csv'...")
    ocorrencias_df_full = pd.read_csv(
        OCORRENCIAS_PATH,
        sep='\t',                     # <<< A MUDANÇA CHAVE! Diz ao pandas para usar tabulação como separador.
        comment='#',
        on_bad_lines='skip',
        low_memory=False,
        encoding='utf-8-sig'
    )
    
    # Agora, selecionamos apenas as colunas que nos interessamos e removemos valores nulos
    ocorrencias_df = ocorrencias_df_full[['decimalLatitude', 'decimalLongitude']].dropna()
    
    print("✅ Arquivo carregado com sucesso!")
    print(f"Total de pontos de ocorrência carregados: {len(ocorrencias_df)}")
    print(ocorrencias_df.head())

except FileNotFoundError:
    print(f"❌ Erro: Arquivo não encontrado em '{OCORRENCIAS_PATH}'. Verifique se o caminho está correto.")
except KeyError:
    print("❌ Erro: As colunas 'decimalLatitude' ou 'decimalLongitude' não foram encontradas no arquivo.")
    print("Verifique os nomes das colunas no cabeçalho do seu CSV.")
except Exception as e:
    print(f"❌ Ocorreu um erro inesperado ao carregar o arquivo: {e}")

# Converter o DataFrame para um GeoDataFrame
gdf_ocorrencias = gpd.GeoDataFrame(
    ocorrencias_df,
    geometry=gpd.points_from_xy(ocorrencias_df.decimalLongitude, ocorrencias_df.decimalLatitude),
    crs="EPSG:4326"
)

# Carregar o mapa do Brasil (shapefile)
shapefile_brasil = glob.glob(os.path.join(BRASIL_SHAPE_PATH, "*.shp"))[0]
brasil_gdf = gpd.read_file(shapefile_brasil)

# Unir todos os estados em um único polígono do Brasil
brasil_poligono = brasil_gdf.unary_union

print("\nMapa do Brasil e pontos de ocorrência carregados com sucesso.")

Tentando carregar o arquivo 'ocorrencias.csv'...
✅ Arquivo carregado com sucesso!
Total de pontos de ocorrência carregados: 15244
   decimalLatitude  decimalLongitude
0          -9.2963          -75.9972
1         -23.4671          -56.4886
2         -23.4671          -56.4886
3         -23.4671          -56.4886
4         -23.4671          -56.4886

Mapa do Brasil e pontos de ocorrência carregados com sucesso.


#### Carregar e empilhar Dados Climáticos Atuais 

In [76]:
# Listar todos os arquivos .tif de clima atual, em ordem alfabética
clima_files = sorted(glob.glob(os.path.join(CLIMA_ATUAL_PATH, "*.tif")))

# Abrir o primeiro arquivo para obter metadados
with rasterio.open(clima_files[0]) as src:
    meta = src.meta

# Atualizar os metadados para o novo raster empilhado (agora com 19 bandas)
meta.update(count=len(clima_files))

# Criar o caminho para o arquivo empilhado
stack_path = os.path.join(CLIMA_ATUAL_PATH, "clima_atual_stack.tif")

# Empilhar os rasters em um único arquivo
with rasterio.open(stack_path, 'w', **meta) as dst:
    for i, file in enumerate(clima_files, 1):
        with rasterio.open(file) as src:
            dst.write(src.read(1), i)

print(f"Rasters climáticos atuais empilhados em: {stack_path}")

Rasters climáticos atuais empilhados em: c:\icev\extensao\abelhas_extensao\data\clima_atual\clima_atual_stack.tif


#### Gerar Dados de Pseudo-Ausência
- Geramos pontos de pseudo-ausência em áreas aleatórias, mas longe dos pontos de presença, para treinar o modelo.

In [77]:
from shapely.geometry import Point

# Número de pontos de pseudo-ausência
num_pseudo_ausencias = len(gdf_ocorrencias) * 2

# Criar uma "zona de exclusão" ao redor dos pontos de presença
buffer_presenca = gdf_ocorrencias.geometry.buffer(0.5).unary_union

# Gerar pontos aleatórios dentro do polígono do Brasil
pseudo_ausencias_points = []
while len(pseudo_ausencias_points) < num_pseudo_ausencias:
    minx, miny, maxx, maxy = brasil_poligono.bounds
    random_point = Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
    if brasil_poligono.contains(random_point) and not buffer_presenca.contains(random_point):
        pseudo_ausencias_points.append(random_point)

# Criar um GeoDataFrame para as pseudo-ausências
gdf_pseudo_ausencias = gpd.GeoDataFrame(
    geometry=pseudo_ausencias_points,
    crs="EPSG:4326"
)

print(f"Gerados {len(gdf_pseudo_ausencias)} pontos de pseudo-ausência.")

Gerados 30488 pontos de pseudo-ausência.


#### Criar um Conjunto de Dados de Treinamento Final

In [78]:
# Função para extrair valores do raster para um GeoDataFrame
def extract_raster_values(gdf, raster_path):
    with rasterio.open(raster_path) as src:
        coords = [(x, y) for x, y in zip(gdf.geometry.x, gdf.geometry.y)]
        values = [val for val in src.sample(coords)]
    return np.array(values)

# Extrair valores para presenças e pseudo-ausências
valores_presenca = extract_raster_values(gdf_ocorrencias, stack_path)
valores_ausencia = extract_raster_values(gdf_pseudo_ausencias, stack_path)

# Criar o dataset final
X = np.vstack((valores_presenca, valores_ausencia))
y = np.array([1] * len(valores_presenca) + [0] * len(valores_ausencia))

# Nomes das features (bio1 a bio19)
feature_names = [os.path.basename(f).split('.')[0] for f in sorted(clima_files)]

# Criar um DataFrame para visualização
df_treinamento = pd.DataFrame(X, columns=feature_names)
df_treinamento['presenca'] = y

print("Conjunto de dados de treinamento criado:")
print(df_treinamento.head())
print(f"\nShape de X: {X.shape}, Shape de y: {y.shape}")

Conjunto de dados de treinamento criado:
         wc2        wc2        wc2     wc2    wc2   wc2        wc2     wc2  \
0  24.271093  24.590834  24.027500  2825.0  363.0  98.0  42.891392  1032.0   
1  23.157845  26.684959  19.153584  1513.0  184.0  57.0  34.639950   520.0   
2  23.157845  26.684959  19.153584  1513.0  184.0  57.0  34.639950   520.0   
3  23.157845  26.684959  19.153584  1513.0  184.0  57.0  34.639950   520.0   
4  23.157845  26.684959  19.153584  1513.0  184.0  57.0  34.639950   520.0   

     wc2    wc2     wc2        wc2        wc2         wc2        wc2  \
0  335.0  719.0  1020.0  11.812813  87.883148   28.844713  30.877750   
1  198.0  474.0   198.0  11.594021  57.734829  317.950592  32.429501   
2  198.0  474.0   198.0  11.594021  57.734829  317.950592  32.429501   
3  198.0  474.0   198.0  11.594021  57.734829  317.950592  32.429501   
4  198.0  474.0   198.0  11.594021  57.734829  317.950592  32.429501   

         wc2        wc2        wc2        wc2  presenca  

### Treinamento do Modelo Random Forest


#### Dividir os Dados e Treinar o Modelo

In [79]:
# Dividir os dados em conjuntos de treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Inicializar o modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced')

# Treinar o modelo com os dados de treino
print("Treinando o modelo Random Forest...")
rf_model.fit(X_train, y_train)
print("Treinamento concluído!")

Treinando o modelo Random Forest...
Treinamento concluído!


#### Avaliar o Modelo

In [80]:
# Fazer previsões no conjunto de teste
y_pred = rf_model.predict(X_test)

# Avaliar a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo no conjunto de teste: {accuracy:.2f}")

# Mostrar um relatório de classificação detalhado
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

# Analisar a importância de cada variável climática
importancias = pd.DataFrame({
    'variavel': feature_names,
    'importancia': rf_model.feature_importances_
}).sort_values('importancia', ascending=False)

print("\nImportância das Variáveis Climáticas:")
print(importancias)

Acurácia do modelo no conjunto de teste: 1.00

Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6098
           1       1.00      0.99      0.99      3049

    accuracy                           1.00      9147
   macro avg       1.00      0.99      1.00      9147
weighted avg       1.00      1.00      1.00      9147


Importância das Variáveis Climáticas:
   variavel  importancia
14      wc2     0.164282
13      wc2     0.130456
2       wc2     0.095245
0       wc2     0.087258
7       wc2     0.067913
11      wc2     0.059742
17      wc2     0.050944
12      wc2     0.047665
16      wc2     0.036975
1       wc2     0.032330
3       wc2     0.032079
4       wc2     0.027911
15      wc2     0.027134
9       wc2     0.026399
18      wc2     0.026199
6       wc2     0.025042
10      wc2     0.024147
5       wc2     0.020117
8       wc2     0.018162


### Previsão para cenários futuros

#### Definir a Função de Previsão

In [81]:
def prever_cenario(cenario_folder_path, modelo, output_path):
    """
    Função para prever a adequabilidade de habitat para um cenário climático futuro.
    """
    print(f"\nProcessando cenário em: {cenario_folder_path}")
    
    cenario_files = sorted(glob.glob(os.path.join(cenario_folder_path, "*.tif")))
    
    with rasterio.open(cenario_files[0]) as src:
        profile = src.profile
        
    raster_data = np.stack([rasterio.open(f).read(1) for f in cenario_files])
    height, width = raster_data.shape[1], raster_data.shape[2]
    raster_data_reshaped = raster_data.reshape((len(cenario_files), -1)).T
    
    # Tratar valores NoData
    nodata_val = -9999.0
    raster_data_reshaped[raster_data_reshaped == nodata_val] = np.nan
    col_mean = np.nanmean(raster_data_reshaped, axis=0)
    inds = np.where(np.isnan(raster_data_reshaped))
    raster_data_reshaped[inds] = np.take(col_mean, inds[1])

    print("Realizando a previsão...")
    previsao = modelo.predict_proba(raster_data_reshaped)[:, 1]
    previsao_mapa = previsao.reshape((height, width))
    
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(previsao_mapa.astype(rasterio.float32), 1)
        
    print(f"Mapa de adequabilidade salvo em: {output_path}")
    return output_path

#### Executar as Previsões para todos os Períodos 

In [82]:
# --- CÉLULA DE DIAGNÓSTICO DAS PASTAS DE CLIMA FUTURO ---

print("--- Diagnóstico das Pastas de Clima Futuro ---")

# Lista dos períodos futuros com os nomes EXATOS das suas pastas
periodos_futuros = [
    "wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2021-2040",
    "wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2041-2060",
    "wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2061-2080",
    "wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2081-2100"
]

clima_futuro_base_path = BASE_DIR / 'data' / 'clima_futuro'

pasta_com_problema = None

for periodo in periodos_futuros:
    pasta_cenario = clima_futuro_base_path / periodo
    print(f"\nVerificando a pasta: {pasta_cenario}")
    
    if not pasta_cenario.exists():
        print(f"❌ ERRO: A pasta '{periodo}' não existe!")
        pasta_com_problema = periodo
        continue
    
    # Listar os arquivos .tif na pasta
    tif_files = sorted(glob.glob(os.path.join(pasta_cenario, "*.tif")))
    
    if not tif_files:
        print(f"❌ ERRO: A pasta '{periodo}' está vazia ou não contém arquivos .tif!")
        pasta_com_problema = periodo
    else:
        print(f"✅ Encontrados {len(tif_files)} arquivos .tif na pasta '{periodo}'.")
        # Opcional: mostrar os 3 primeiros arquivos
        print(f"   Exemplos: {os.path.basename(tif_files[0])}, {os.path.basename(tif_files[1])}, {os.path.basename(tif_files[2])}")

if pasta_com_problema:
    print("\n--- SOLUÇÃO ---")
    print(f"A pasta '{pasta_com_problema}' está com problemas.")
    print("Para corrigir:")
    print(f"1. Encontre o arquivo .zip original do WorldClim correspondente a '{pasta_com_problema}'.")
    print("2. Apague a pasta vazia (ou com problemas) de 'data/clima_futuro/'.")
    print("3. Extraia novamente o conteúdo do .zip para a pasta 'data/clima_futuro/'.")
else:
    print("\n✅ Todas as pastas de cenário futuro parecem estar OK!")


--- Diagnóstico das Pastas de Clima Futuro ---

Verificando a pasta: c:\icev\extensao\abelhas_extensao\data\clima_futuro\wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2021-2040
❌ ERRO: A pasta 'wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2021-2040' não existe!

Verificando a pasta: c:\icev\extensao\abelhas_extensao\data\clima_futuro\wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2041-2060
❌ ERRO: A pasta 'wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2041-2060' não existe!

Verificando a pasta: c:\icev\extensao\abelhas_extensao\data\clima_futuro\wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2061-2080
❌ ERRO: A pasta 'wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2061-2080' não existe!

Verificando a pasta: c:\icev\extensao\abelhas_extensao\data\clima_futuro\wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2081-2100
❌ ERRO: A pasta 'wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2081-2100' não existe!

--- SOLUÇÃO ---
A pasta 'wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2081-2100' está com problemas.
Para corrigir:
1. Encontre o arquivo .zip original do WorldClim correspondente a 'wc2.1_10m_bi

In [ ]:
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import os
import glob

def detectar_mapas_brasil():
    """Detecta automaticamente os mapas mascarados do Brasil"""
    mapas_encontrados = {}
    
    # Procurar mapas de distribuição atual
    atual_files = glob.glob("mapa_predito_atual_*_brasil.tif")
    if atual_files:
        mapas_encontrados["atual"] = atual_files[0]
    
    # Procurar mapas de distribuição futura
    futuro_files = glob.glob("mapa_predito_futuro*_brasil.tif")
    if futuro_files:
        mapas_encontrados["futuro"] = futuro_files[0]
    
    # Procurar mapas de mudança
    mudanca_files = glob.glob("mapa_mudanca_*_brasil.tif")
    if mudanca_files:
        mapas_encontrados["mudanca"] = mudanca_files[0]
    
    return mapas_encontrados

def visualizar_mapa(tif_path, titulo, salvar=True, escala_padronizada=True):
    with rasterio.open(tif_path) as src:
        data = src.read(1).astype(float)

        # Trata valores nodata
        if src.nodata is not None:
            data[data == src.nodata] = np.nan

        # Obtém valores mínimo e máximo reais para estatísticas
        valid_data = data[~np.isnan(data)]
        if len(valid_data) == 0:
            print(f"❌ Nenhum dado válido encontrado em {titulo}")
            return
            
        v_min_real = np.nanmin(valid_data)
        v_max_real = np.nanmax(valid_data)
        
        # Calcula estatísticas básicas
        mean = np.nanmean(valid_data)
        std = np.nanstd(valid_data)
        
        print(f"📊 Estatísticas para {titulo}:")
        print(f"   - Min: {v_min_real:.4f}, Max: {v_max_real:.4f}, Média: {mean:.4f}, Desvio: {std:.4f}")
        
        # Define escala de visualização
        if escala_padronizada:
            if "mudança" in titulo.lower() or "mudanca" in titulo.lower():
                # Para mapas de mudança: escala simétrica baseada no máximo absoluto global
                abs_max = max(abs(v_min_real), abs(v_max_real))
                # Usar uma escala padrão de -0.8 a +0.8 para mudanças
                v_min, v_max = -0.8, 0.8
                cmap = plt.cm.RdBu_r  # Vermelho = perda, Azul = ganho
                label = 'Mudança na Probabilidade'
                print(f"   📏 Escala padronizada para mudança: {v_min} a {v_max}")
            else:
                # Para mapas de probabilidade: escala padronizada 0-1
                v_min, v_max = 0.0, 1.0
                cmap = plt.cm.viridis
                label = 'Probabilidade de Adequabilidade'
                print(f"   📏 Escala padronizada para probabilidade: {v_min} a {v_max}")
        else:
            # Escala dinâmica baseada nos dados
            v_min, v_max = v_min_real, v_max_real
            if "mudança" in titulo.lower() or "mudanca" in titulo.lower():
                cmap = plt.cm.RdBu_r
                label = 'Mudança na Probabilidade'
            else:
                cmap = plt.cm.viridis
                label = 'Probabilidade de Adequabilidade'
            print(f"   📏 Escala dinâmica: {v_min:.4f} a {v_max:.4f}")
        
        # Configura visualização
        plt.figure(figsize=(12, 10))
        
        # Cria o mapa
        im = plt.imshow(data, cmap=cmap, vmin=v_min, vmax=v_max)
        
        # Adiciona título e legenda
        plt.title(titulo, fontsize=16, pad=20)
        plt.axis('off')
        cbar = plt.colorbar(im, label=label, shrink=0.7)
        cbar.ax.tick_params(labelsize=12)
        
        # Adiciona informações sobre valores reais e escala
        info_text = f"Valores reais: {v_min_real:.4f} a {v_max_real:.4f}\nMédia: {mean:.4f} ± {std:.4f}"
        if escala_padronizada:
            info_text += f"\nEscala padronizada: {v_min} a {v_max}"
        
        plt.annotate(info_text, 
                     xy=(0.02, 0.02), xycoords='figure fraction', 
                     fontsize=11, color='black', backgroundcolor='white',
                     bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
        
        plt.tight_layout()
        
        # Salva o mapa em alta resolução para o trabalho acadêmico
        if salvar:
            nome_arquivo = f"{titulo.replace(' ', '_').replace('(', '').replace(')', '')}.png"
            plt.savefig(nome_arquivo, dpi=300, bbox_inches='tight')
            print(f"💾 Mapa salvo como: {nome_arquivo}")
        
        plt.show()

def visualizar_mapa_diferenca(atual_path, futuro_path, escala_padronizada=True):
    """Cria um mapa de diferença entre distribuição futura e atual"""
    
    if not os.path.exists(atual_path) or not os.path.exists(futuro_path):
        print("❌ Arquivos de mapa não encontrados para calcular diferença")
        return
    
    with rasterio.open(atual_path) as src_atual, rasterio.open(futuro_path) as src_futuro:
        data_atual = src_atual.read(1).astype(float)
        data_futuro = src_futuro.read(1).astype(float)
        
        # Trata valores nodata
        if src_atual.nodata is not None:
            data_atual[data_atual == src_atual.nodata] = np.nan
        if src_futuro.nodata is not None:
            data_futuro[data_futuro == src_futuro.nodata] = np.nan
        
        # Calcula diferença
        data_diff = data_futuro - data_atual
        
        # Configurar visualização
        plt.figure(figsize=(12, 10))
        
        # Paleta divergente para mudanças (vermelho = perda, azul = ganho)
        cmap = plt.cm.RdBu_r
        
        # Determinar limite para visualização
        valid_diff = data_diff[~np.isnan(data_diff)]
        if len(valid_diff) == 0:
            print("❌ Nenhum dado válido para calcular diferença")
            return
        
        v_min_real = np.nanmin(data_diff)
        v_max_real = np.nanmax(data_diff)
        
        if escala_padronizada:
            # Escala padronizada para mudanças
            v_min, v_max = -0.8, 0.8
        else:
            # Escala dinâmica simétrica
            abs_max = max(abs(v_min_real), abs(v_max_real))
            v_min, v_max = -abs_max, abs_max
        
        im = plt.imshow(data_diff, cmap=cmap, vmin=v_min, vmax=v_max)
        
        plt.title("Mudança na Distribuição (Futuro - Atual)", fontsize=16, pad=20)
        plt.axis('off')
        cbar = plt.colorbar(im, label='Mudança na Probabilidade', shrink=0.7)
        cbar.ax.tick_params(labelsize=12)
        
        # Calcular estatísticas da diferença
        mean_diff = np.nanmean(data_diff)
        perc_loss = np.sum(data_diff < -0.1) / np.sum(~np.isnan(data_diff)) * 100
        perc_gain = np.sum(data_diff > 0.1) / np.sum(~np.isnan(data_diff)) * 100
        
        # Adiciona informações sobre valores
        info_text = (f"Valores reais: {v_min_real:.4f} a {v_max_real:.4f}\n"
                     f"Média da mudança: {mean_diff:.4f}\n"
                     f"Área com perda (< -0.1): {perc_loss:.1f}%\n"
                     f"Área com ganho (> 0.1): {perc_gain:.1f}%")
        
        if escala_padronizada:
            info_text += f"\nEscala padronizada: {v_min} a {v_max}"
        
        plt.annotate(info_text, xy=(0.02, 0.02), xycoords='figure fraction', 
                     fontsize=11, color='black', backgroundcolor='white',
                     bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
        
        plt.tight_layout()
        
        # Salva o mapa
        plt.savefig("Mudanca_na_Distribuicao.png", dpi=300, bbox_inches='tight')
        print(f"💾 Mapa de mudança salvo como: Mudanca_na_Distribuicao.png")
        
        plt.show()

if __name__ == "__main__":
    # Detectar mapas automaticamente
    mapas = detectar_mapas_brasil()
    
    if not mapas:
        print("❌ Nenhum mapa mascarado encontrado!")
        print("Execute primeiro:")
        print("1. python gerar_mapas_ajustados.py")
        print("2. python mascarar_brasil.py")
        exit(1)
    
    print("📂 Mapas detectados:")
    for tipo, arquivo in mapas.items():
        print(f"  - {tipo}: {arquivo}")
    
    # Perguntar sobre padronização
    print("\n🎨 Escolha o tipo de escala para visualização:")
    print("1. Escala padronizada (0-1 para probabilidades, -0.8 a +0.8 para mudanças)")
    print("2. Escala dinâmica (baseada nos valores dos dados)")
    opcao_escala = input("Opção (1 ou 2) [padrão=1]: ").strip() or "1"
    
    escala_padronizada = opcao_escala == "1"
    
    if escala_padronizada:
        print("✅ Usando escalas padronizadas para comparação científica")
    else:
        print("✅ Usando escalas dinâmicas para visualização detalhada")
    
    # Visualizar mapas de distribuição
    if "atual" in mapas:
        print(f"\n📍 Exibindo: Distribuição Atual")
        # Extrair sufixo do arquivo para identificar o cenário
        nome_arquivo = os.path.basename(mapas["atual"])
        if "futuro2" in nome_arquivo:
            sufixo = " (cenário futuro2)"
        elif "futuro3" in nome_arquivo:
            sufixo = " (cenário futuro3)"
        else:
            sufixo = ""
        visualizar_mapa(mapas["atual"], f"Distribuição Atual{sufixo}", escala_padronizada=escala_padronizada)
    
    if "futuro" in mapas:
        print(f"\n📍 Exibindo: Distribuição Futura")
        # Extrair sufixo do arquivo para identificar o cenário
        nome_arquivo = os.path.basename(mapas["futuro"])
        if "futuro2" in nome_arquivo:
            sufixo = " (cenário futuro2)"
        elif "futuro3" in nome_arquivo:
            sufixo = " (cenário futuro3)"
        else:
            sufixo = ""
        visualizar_mapa(mapas["futuro"], f"Distribuição Futura{sufixo}", escala_padronizada=escala_padronizada)
    
    # Visualizar mapa de mudança se disponível
    if "mudanca" in mapas:
        print(f"\n📍 Exibindo: Mapa de Mudança")
        nome_arquivo = os.path.basename(mapas["mudanca"])
        if "futuro2" in nome_arquivo:
            sufixo = " (cenário futuro2)"
        elif "futuro3" in nome_arquivo:
            sufixo = " (cenário futuro3)"
        else:
            sufixo = ""
        visualizar_mapa(mapas["mudanca"], f"Mudança na Distribuição{sufixo}", escala_padronizada=escala_padronizada)
    
    # Se temos atual e futuro, calcular diferença manualmente se não existe mapa de mudança
    elif "atual" in mapas and "futuro" in mapas:
        print(f"\n📍 Calculando mapa de diferença entre distribuição futura e atual")
        visualizar_mapa_diferenca(mapas["atual"], mapas["futuro"], escala_padronizada=escala_padronizada)


❌ Nenhum mapa mascarado encontrado!
Execute primeiro:
1. python gerar_mapas_ajustados.py
2. python mascarar_brasil.py
📂 Mapas detectados:

🎨 Escolha o tipo de escala para visualização:
1. Escala padronizada (0-1 para probabilidades, -0.8 a +0.8 para mudanças)
2. Escala dinâmica (baseada nos valores dos dados)
